In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

GOOGLE_API_KEY=os.environ["GOOGLE_API_KEY"] = "your api key here"

if GOOGLE_API_KEY:
    print("Gemini API key setup complete.")
else:
    print("Error: GOOGLE_API_KEY not found in environment variables.")

আপনার API Key সফলভাবে লোড হয়েছে।


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro")



অবশ্যই! আজকে বাংলাদেশের সার্বিক অবস্থা সম্পর্কে একটি ধারণা নিচে দেওয়া হলো। বিষয়টি কয়েকটি ভাগে ভাগ করে আলোচনা করলে বুঝতে সুবিধা হবে:

**১. অর্থনৈতিক অবস্থা (Economic Situation):**

এটি বর্তমানে সবচেয়ে আলোচিত বিষয়। বাংলাদেশের অর্থনীতি একটি মিশ্র পরিস্থিতির মধ্যে দিয়ে যাচ্ছে।

*   **চ্যালেঞ্জসমূহ (Challenges):**
    *   **উচ্চ মূল্যস্ফীতি (High Inflation):** নিত্যপ্রয়োজনীয় জিনিসপত্রের দাম অনেক বেশি, যা সাধারণ মানুষের জীবনযাত্রার ওপর ব্যাপক চাপ সৃষ্টি করেছে। চাল, ডাল, তেল থেকে শুরু করে প্রায় সবকিছুর দামই ঊর্ধ্বমুখী।
    *   **বৈদেশিক মুদ্রার রিজার্ভ (Foreign Exchange Reserves):** রিজার্ভ কমে যাওয়া একটি বড় উদ্বেগের কারণ। আমদানি ব্যয় মেটাতে এবং ডলারের বিপরীতে টাকার মান স্থিতিশীল রাখতে সরকারকে বেশ বেগ পেতে হচ্ছে।
    *   **জ্বালানি ও বিদ্যুৎ সংকট (Energy & Power Crisis):** যদিও পরিস্থিতি আগের চেয়ে কিছুটা উন্নত, তবুও মাঝে মাঝে লোডশেডিং এবং জ্বালানির মূল্যবৃদ্ধির প্রভাব অর্থনীতিতে পড়ছে।
    *   **ব্যাংকিং খাত:** খেলাপি ঋণ এবং কিছু ব্যাংকের আর্থিক দুর্বলতা নিয়েও উদ্বেগ রয়েছে।

*   **ইতিবাচক দিক 

In [107]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro")

In [108]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

In [131]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("G:\Agent\search__agent\profile2.pdf")

docs = loader.load()


<>:2: SyntaxWarning: invalid escape sequence '\A'
<>:2: SyntaxWarning: invalid escape sequence '\A'
C:\Users\User\AppData\Local\Temp\ipykernel_14792\584753482.py:2: SyntaxWarning: invalid escape sequence '\A'
  loader = PyPDFLoader("G:\Agent\search__agent\profile2.pdf")


In [132]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,  # chunk size (characters)
    chunk_overlap=20,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

Split blog post into 28 sub-documents.


In [133]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

In [134]:
from langchain_community.vectorstores import FAISS
vector = FAISS.from_documents(all_splits, embeddings)

In [135]:
retriever = vector.as_retriever()

In [136]:
document_ids = vector.add_documents(documents=all_splits)

print(document_ids[:3])

['dd33d62d-de70-499d-a7ef-6c6eca7c9d82', '5f0b8601-b36f-4ac8-82ef-5206333f5081', 'f4ad90a8-f5ba-486e-9718-0316e5d863fd']


In [137]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [138]:
from langchain.agents.middleware import dynamic_prompt, ModelRequest
from langchain.agents import create_agent
@dynamic_prompt
def prompt_with_context(request: ModelRequest) -> str:
    """Inject context into state messages."""
    last_query = request.state["messages"][-1].text
    retrieved_docs = vector.similarity_search(last_query)

    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

    system_message = (
        "You are a helpful assistant. Use the following context in your response:"
        f"\n\n{docs_content}"
    )

    return system_message


agent = create_agent(llm, tools=[], middleware=[prompt_with_context])

In [141]:
query = "সার ব্যবস্থাপনা?"
for step in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

সার ব্যবস্থাপনা?
================================== Ai Message ==================================

প্রদত্ত তথ্য অনুসারে, **সার ব্যবস্থাপনা (কেজি/বিঘা)** সম্পর্কে বলা হয়েছে।

এর মানে হলো, প্রতি বিঘা জমিতে কেজি (kg) এককে কী পরিমাণ সার প্রয়োগ করতে হবে, তার নির্দেশনা নিচে দেওয়া থাকবে।

তবে, আপনার দেওয়া অংশে শুধু শিরোনামটিই রয়েছে, সারের নাম বা পরিমাণ উল্লেখ করা নেই। আপনি সম্পূর্ণ তথ্য দিলে, বিস্তারিত জানানো সম্ভব হবে।
